In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime as dt
import pandas as pd
import sqlalchemy
import os

In [15]:
# pick a backend - Django
# DB - sqlite
# electron.js - package for building desktop applications that works across os's


def message_path():
    wd = os.getcwd()
    split_direct = wd.split("/")
    db_path = '/'.join(split_direct[1:3]) + '/Library/Messages/chat.db'
    return db_path, split_direct



class Query:
    # Class Variables for querying
    path, split_direct = message_path()
    # print(path, split_direct)
    engine = sqlalchemy.create_engine("sqlite:////" + path, pool_pre_ping=True)
    Session = sessionmaker(bind=engine)
    Base = automap_base()
    Base.prepare(engine, reflect=True)
    
    # tables 
    message = Base.classes.message
    chat = Base.classes.chat
    chat_message_join = Base.classes.chat_message_join

    # need to build a flexible date filter which can take in different formats ?
    def __init__(self, date_start = '2006-01-01', date_end = dt.today()):
        self.date_start = date_start
        self.date_end = date_end


    def base_query(self, sesh = Session, message = message, chat = chat, chat_message_join = chat_message_join):
    # clean up date .strftime('%Y-%m-%d %H:%M:%S')
        q = sesh().query(chat.chat_identifier,
                chat.service_name,
                chat.display_name,
                message.date,
                message.text,
                message.is_from_me,                         
                message.date_read, # applicable for is_from_me and ppl who have read receipts
                message.date_delivered,
                message.was_delivered_quietly,
                message.did_notify_recipient).join(chat_message_join, 
                     chat_message_join.chat_id == chat.ROWID).join(chat_message_join.message_id == message.ROWID)
        
        return q

    
    
    
    
def execute(query):
    return pd.read_sql(query.statement, query.session.bind)
        

In [16]:
q = Query()
q = q.base_query()



In [17]:
execute(q)

ArgumentError: Expected mapped entity or selectable/table as join target

In [14]:
print(str(q))


ArgumentError: Expected mapped entity or selectable/table as join target